# 01.06 Classwork №3: русскоязычный корпус и перплексия

## Сбор корпуса

Будем парсить Медузу, 2 рубрики: новости и шапито

In [10]:
import lxml.html
import json
import requests

site_url = 'https://meduza.io/api/v3/search?chrono='
api_key='&page='
api_key_2='&per_page=20&locale=ru'
categories = ['news', 'shapito']

corpus = {} #{category:articles[]}
for category in categories: 
    articles = []
    for page in range(0,50):    
        #print (site_url+category+api_key+str(page)+api_key_2)
        r = requests.get(site_url+category+api_key+str(page)+api_key_2)
        content_string = r.content.decode("utf-8")
        content_json = json.loads(content_string)
        for item in content_json['documents']:
            full_text_url='https://meduza.io/api/v3/'+content_json['documents'][item]['url']
            r2 = requests.get(full_text_url)
            text_string = r2.content.decode("utf-8")  
            full_text_json = json.loads(text_string)
            text = full_text_json['root']['content']['body']
            document = lxml.html.document_fromstring(text)
            cleantext = document.text_content()
            articles.append([cleantext])
    corpus[category] = articles

In [11]:
for k,v in corpus.items():
    print (k, len(v))

shapito 1000
news 1000


In [12]:
print(len(corpus['news']))

1000


## Триграммы из НКРЯ

In [8]:
def load_ngrams(in_file):
    input_f = open(in_file, "r", encoding="utf8")
    n_dict = {}
    for line in input_f:
        line = line.replace('\n','').split('\t')
        n_dict[tuple(line[0:])] = int(line[0])
    input_f.close()
    return n_dict

In [2]:
n2_dict = load_ngrams("2grams-3.txt")
print (len(n2_dict.keys()))

6750525


In [9]:
n3_dict = load_ngrams("3grams-3.txt")
print (len(n3_dict.keys()))

4655170


## Перплексия

In [100]:
import math
def compute_perplexity(category, n3_dict, n2_dict):
    sum_probability = 0
    unknown_words = 0
    min_probability = 1
    sum_log = 0

    for i in range(2, len(category)):
        try:
            current_probability = (n3_dict[(category[i-2], category[i-1], category[i])] + 0.0) 
            / n2_dict[(category[i-2],category[i-1])]
            sum_probability += current_probability
            if current_probability < min_probability:
                min_probability = current_probability
        except KeyError:
            unknown_words +=1
    print (unknown_words, len(category), sum_probability)
    known_words = len(category) - unknown_words
    min_probability /= 10
    probability_to_substract = (unknown_words * min_probability + 0.0) / known_words
    for i in range(2, len(category)):
        try:
            current_probability = (n3_dict[(category[i-2], category[i-1], category[i])] + 0.0) 
            / n2_dict[(category[i-2],category[i-1])]
            sum_log += math.log(current_probability - probability_to_substract, 2)
        except KeyError:
            sum_log += math.log(min_probability, 2)

    return sum_log

In [52]:
import nltk
import string
news_list = corpus['news']
news = []
for text in news_list:
    text_str = ''.join(text)
    line = nltk.word_tokenize(text_str)
    line = [word.encode("utf-8").decode("utf-8").lower() for word in line if word not in string.punctuation
                    and word not in [u'\u2012', u'\u2013', u'\u2014', u'\u2015', u'``', u"''"]]
    news += line

In [103]:
sum_log_2 = compute_perplexity(news, n3_dict, n2_dict)
print("Perplexity for news = ",2**(-sum_log_2 / len(news)))

288530 288532 0
Perplexity for news =  9.999840394239554


In [104]:
import nltk
import string
shapito_list = corpus['shapito']
shapito = []
for text in shapito_list:
    text_str = ''.join(text)
    line = nltk.word_tokenize(text_str)
    line = [word.encode("utf-8").decode("utf-8").lower() for word in line if word not in string.punctuation
                    and word not in [u'\u2012', u'\u2013', u'\u2014', u'\u2015', u'``', u"''"]]
    shapito += line

In [105]:
sum_log_1 = compute_perplexity(shapito, n3_dict, n2_dict)
print("Perplexity for shapito = ",2**(-sum_log_2 / len(shapito)))

179870 179872 0
Perplexity for shapito =  40.18695746319039
